# Gastric Warriors Differential gene expression analysis



Importing correct files: Using bash

For running the jupyter make sure your in the correct directory: /cephyr/NOBACKUP/groups/bbt045_2025/groups/group_gaswar/


In [ ]:
%%bash

mkdir -p Data/
workdir="/cephyr/NOBACKUP/groups/bbt045_2025/groups/group_gaswar/Data/"
echo "Working directory set to $workdir"

RNAlist="/cephyr/NOBACKUP/groups/bbt045_2025/Projects/Data/RNA-seq-list.txt"
RNAdata="/cephyr/NOBACKUP/groups/bbt045_2025/Projects/Data/RNAseq/"
echo "RNAseq-data in ${RNAdata}"

#extract the rows that contain the healthconditions and store the patient name (column 1) in a textfile
grep "Met" $RNAlist | cut -f 1 > patients.txt
grep "Hp" $RNAlist | cut -f 1 >> patients.txt

#find the fastq-files that correspond to patient names and copy to the data-folder

find $RNAdata -type f -name "*_1.fastq.gz" | grep -Ff patients.txt | xargs -I {} cp {} "$workdir"
find $RNAdata -type f -name "*_2.fastq.gz" | grep -Ff patients.txt | xargs -I {} cp {} "$workdir"

Create a CSV with patient list and health condition, which will be used later

In [ ]:
%%bash

#Create patient list
RNAlist="/cephyr/NOBACKUP/groups/bbt045_2025/Projects/Data/RNA-seq-list.txt"

# Define the output file
OUTPUT="patient_list.csv"
# Add CSV header
echo "PatientNumber,HealthCondition" > $OUTPUT

# Create patient_list.txt with "Met" and "Hp" conditions
awk '{print $1, ",Met"}' <(grep "Met" $RNAlist | cut -f1) >> $OUTPUT
awk '{print $1, ",Hp"}' <(grep "Hp" $RNAlist | cut -f1) >> $OUTPUT


Now decompress all the files

In [ ]:
%%bash
#set the working directory
workdir="/cephyr/NOBACKUP/groups/bbt045_2025/groups/group_gaswar/Data/"

#use gzip to decrompress the files
gzip -d "${workdir}"*

# MultiQC

In [ ]:
#First generate fastQC reports, all files in Data and output to /subset_fastqc/
! mkdir -p before_trim_fastqc
! fastqc Data/*.fastq -o before_trim_fastqc/

In [ ]:
#Take entire subset_fastqc/ and generate multiqc-file
! multiqc before_trim_fastqc/ -n multiqc_before_trim.html

# Preprocessing using FastP
> 

In [ ]:
%%bash

#Output
mkdir -p trimmed_fastq


# Loop through all forward reads (_R1.fastq.gz.subset.fastq) and find their corresponding reverse reads (_R2.fastq.gz.subset.fastq)
for R1 in Data/*_1.fastq; do
    # Extract the sample name by removing _R1.fastq.gz.subset.fastq
    SAMPLE=$(basename "$R1" _1.fastq)
    # Define file names
    R2="Data/${SAMPLE}_2.fastq"         # Reverse read file

    echo "Checking: $R1 and $R2"

    OUT_R1="trimmed_fastq/${SAMPLE}_1_trimmed_fastq.gz"
    OUT_R2="trimmed_fastq/${SAMPLE}_2_trimmed_fastq.gz"
    # Run fastp, input for -i and -I (Forward and Reverse reads), output into OUT_ R1 and R2. Filter scores under 30, and better trimming with cut_right and cut_front
    fastp -i "$R1" -I "$R2" -o "$OUT_R1" -O "$OUT_R2" \
        --qualified_quality_phred 30 \
        --cut_right \
        --cut_right_window_size 4 \
        --cut_right_mean_quality 30 \
        --cut_front 

    echo "Finished processing: $SAMPLE"
done

# FastQC and multiQC on processed reads

In [ ]:
#Fastqc of the fastp dataset
#create directory for the fastqc files
! mkdir -p FastQC_aftertrim
! fastqc trimmed_fastq/*_fastq.gz -o FastQC_aftertrim/
# multiQC on the entire directory 
! multiqc FastQC_aftertrim/ -n aftertrim_multiqc.html

# Building STAR reference genome
Genome from https://www.ensembl.org/Homo_sapiens/Info/Index 

In [ ]:
%%bash

gunzip Reference/*.gz 

mkdir -p human_index

STAR --runMode genomeGenerate --genomeDir human_index/ --genomeFastaFiles Reference/Homo_sapiens.GRCh38.dna_sm.primary_assembly.fa \
    --sjdbGTFfile Reference/Homo_sapiens.GRCh38.113.gtf --runThreadN 8

Getting the genes names from GTF file

In [ ]:
# Extract `gene_id` and `gene_name` from a GTF file and create a clean mapping file.
# Remove everything before and including 'gene_id "' using sed
# Replace the text from the first quote (`"`) after `gene_id` until `gene_name "` with a tab (`\t`)
# Remove everything from the remaining quote (`"`) onward
# Sort the lines alphabetically and remove duplicate entries
# The final output is a tab-separated file (`gene_id_name.mapping.txt`) with unique gene mappings
! sed "s/.*gene_id .//" Homo_sapiens.GRCh38.113.gtf | sed "s/\".*gene_name ./\t/" | sed "s/\".*//" | sort | uniq > gene_id_name.mapping.txt

# Sort out the files from featureCounts
Get a nice datafram in csv file

In [ ]:
import pandas as pd

# Load featureCounts file, skipping header lines (lines starting with "#")
gene_counts_file = "gene_counts.txt"
df = pd.read_csv(gene_counts_file, sep="\t", comment="#")

# Rename the columns by extracting patient names from file paths
df.columns = df.columns.str.replace(r'.*/JOB_TMP/(\w+)_Aligned.sortedByCoord.out.bam', r'\1', regex=True)

# Set Geneid as index
df.set_index("Geneid", inplace=True)


df.to_csv("gene_counts_cleaned.csv")

counts = df.drop(columns=['Chr','Start','End','Length','Strand'])

# Save as CSV for later use
counts.to_csv("counts_data.csv")

# Display the first few row
print(counts.head())


# Reove genes that are low expressed

In [ ]:
import pandas as pd

# Load data
gene_counts = "counts_data.csv"
counts = pd.read_csv(gene_counts)

# Ensure only numeric columns are summed
counts_numeric = counts.select_dtypes(include="number")  

# Filter rows where sum > 20
counts = counts.loc[counts_numeric.sum(axis=1) > 50, :]

# Save the filtered dataset
counts.to_csv("counts.filtered.csv", index=False)  # index=False prevents writing row indices

# PCA

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# Load gene expression data
gene_counts = pd.read_csv("counts.filtered.csv")
gene_counts = gene_counts.set_index("Geneid").T  # Transpose: Patients as rows, Genes as columns
gene_counts.index.name = "PatientNumber"  # Rename index for reference

# Load patient conditions data
conditions = pd.read_csv("patient_list.csv")

# Ensure patient IDs match (strip spaces and set types)
gene_counts.index = gene_counts.index.astype(str).str.strip()
conditions["PatientNumber"] = conditions["PatientNumber"].astype(str).str.strip()

# Extract patient names and condition into a dictionary, zip pairs the condition and patient together
condition_map = dict(zip(conditions["PatientNumber"], conditions["HealthCondition"]))
# Map the conditions to the index names (patient names) in gene counts
condition_labels = gene_counts.index.map(condition_map)  
print(condition_labels) # A list with all the conditions

# Convert gene expression data to numeric and fill NaNs
X = gene_counts.apply(pd.to_numeric, errors="coerce").fillna(0)

# Standardize gene expression values, due to PCA assuming data is normally distrubuted
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Apply PCA
pca = PCA(n_components=2)
pca_result = pca.fit_transform(X_scaled)
var1, var2 = pca.explained_variance_ratio_*100

# Convert PCA output to DataFrame
pca_df = pd.DataFrame(data=pca_result, columns=["PC1", "PC2"])
pca_df["Condition"] = condition_labels.values  # Add conditions
pca_df["PatientNumber"] = gene_counts.index  # Add patient labels

# Define colors for conditions
color_map = {"Met": "red", "Hp": "blue"}
colors = pca_df["Condition"].map(color_map)

# Plot PCA results
plt.figure(figsize=(8,6))
for condition in pca_df["Condition"].unique():
    subset = pca_df[pca_df["Condition"] == condition]
    plt.scatter(subset["PC1"], subset["PC2"], label=condition, c=color_map[condition], s=100, edgecolors="k")
    #Add the patient numbers to the plot
    for i, row in subset.iterrows():
        plt.text(row["PC1"], row["PC2"] - 5, row["PatientNumber"], fontsize=8, ha='left', va='top')
plt.title("PCA of Gene Expression Data")
plt.xlabel(f"Principal Component 1, Variance explained: {var1:.1f}%")
plt.ylabel(f"Principal Component 2, Variance explained: {var2:.1f}%")
plt.legend(title="Health Condition")
plt.grid(True)

plt.savefig("pca_plot.pdf", format="pdf", dpi=300, bbox_inches="tight") #Save pdf

plt.show()


# DESeq2

In [ ]:
#install rpy2 in order for R to work
!pip install rpy2==3.5.11
import rpy2
%reload_ext rpy2.ipython

In [ ]:
%%R
#source the r file
source("/cephyr/NOBACKUP/groups/bbt045_2025/groups/group_gaswar/DESeq2.r")


In [ ]:
%%R
file.info("volcano_plot.pdf")$size

# Extract geneIDs and check gene names
Store the result as a csv file which was imported into excel for a better overview

In [ ]:
import pandas as pd

gene_id_mapping = "Reference/gene_id_name.mapping.txt"
DESeq_results = "sorted_deseq_results.csv"

gene_id = pd.read_csv(gene_id_mapping, sep = "\t", comment="#", header = None)
#add column names
gene_id.columns = ["GeneID","Gene"]
DESeq2 = pd.read_csv(DESeq_results, sep = ",")
DESeq2.columns = ["GeneID", "BaseMean", "Log2FoldChange", "Adjusted_p-value"]



#print(gene_id.iloc[:,0])
#print(gene_id.head())
#print(DESeq2.head())
# List to store matched gene names
gene_names_list = []

for index, row in DESeq2.iterrows():
    GeneID = row["GeneID"]

    # Matching the gene IDs to then extract the Gene name
    match = gene_id[gene_id["GeneID"] == GeneID]["Gene"]

    # Append the first matched gene name, if it doesnt get matched then add "Unknown"
    if not match.empty:
        gene_names_list.append(match.iloc[0])  # Get the first match
    else:
        gene_names_list.append("Unknown")

# Adding gene_names_list to original dataframe
DESeq2["Gene_names"] = gene_names_list

DESeq2.to_csv("Significant_genes.csv", index=False)

# Comparing Metaplasia samples based on infection
Differential expression analysis comparing infected vs non-infected metaplasia patients.

In [ ]:
!pip install rpy2==3.5.11
import rpy2
%reload_ext rpy2.ipython

In [ ]:
%%R
#source the r file
source("/cephyr/NOBACKUP/groups/bbt045_2025/groups/group_gaswar/met_patient.r")

Store results as csv file and associate genes names to the gene IDs

In [ ]:
import pandas as pd

gene_id_mapping = "Reference/gene_id_name.mapping.txt"
DESeq_results = "compare_met_results.csv"

gene_id = pd.read_csv(gene_id_mapping, sep = "\t", comment="#", header = None)
gene_id.columns = ["GeneID","Gene"]
DESeq2 = pd.read_csv(DESeq_results, sep = ",")
DESeq2.columns = ["GeneID", "BaseMean", "Log2FoldChange", "Adjusted_p-value"]

# List to store matched gene names
gene_names_list = []

for index, row in DESeq2.iterrows():
    GeneID = row["GeneID"]

    # Matching the gene IDs to then extract the Gene name
    match = gene_id[gene_id["GeneID"] == GeneID]["Gene"]

    # Append the first matched gene name, if it doesnt get matched then add "Unknown"
    if not match.empty:
        gene_names_list.append(match.iloc[0])  # Get the first match
    else:
        gene_names_list.append("Unknown")

# Adding gene_names_list to original dataframe
DESeq2["Gene_names"] = gene_names_list

DESeq2.to_csv("Significant_met_genes.csv", index=False)

The CSV-files were further explored in excel